In [8]:
import fastai
from fastai.vision import *
from fastai.callbacks import *
from fastai.torch_core import *
from fastai.callback import *
from fastai.basic_train import *
import torchvision.models as tmodels

import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from sklearn.metrics import accuracy_score
from scipy.misc import imsave, imread

import os
import sys

sys.path.insert(0, '/home/santhosr/Documents/Birad/FastAI/RaceClassification/' )

from utils import *

inputFolder1 = '/home/santhosr/Documents/Birad/ProcessedData/FullRes'
truthFile1 = '/home/santhosr/Documents/Birad/birad_targetFile.csv'

inputFolder2 = '/home/santhosr/Documents/Birad/ProcessedData/PennExtra_3500/'
truthFile2 = '/home/santhosr/Documents/Birad/RaceDL_ExtraCaucasian.csv'

df1 = pd.read_csv('/home/santhosr/Documents/Birad/birad_targetFile.csv')
df1.drop(['PresIntentType','DBT'],inplace = True,axis=1)


df2 = pd.read_csv('/home/santhosr/Documents/Birad/RaceDL_ExtraCaucasian.csv')
df2.Medview_Race = 'White'
truth = pd.concat([df1,df2],sort=True)

In [2]:
df1.Medview_Race.value_counts()

African American         10369
White                     2416
Other                      214
Asian                      205
Unknown                     67
Hispanic Latino/White       51
Hispanic Latino/Black       22
East Indian                 16
Pacific Island               6
American Indian              5
Name: Medview_Race, dtype: int64

In [3]:
df2.Medview_Race.value_counts()

White    3500
Name: Medview_Race, dtype: int64

In [4]:
truth.Medview_Race.value_counts()

African American         10369
White                     5916
Other                      214
Asian                      205
Unknown                     67
Hispanic Latino/White       51
Hispanic Latino/Black       22
East Indian                 16
Pacific Island               6
American Indian              5
Name: Medview_Race, dtype: int64

In [5]:
truth = truth.drop_duplicates(subset=['DummyID'])

In [6]:
truth.Medview_Race.value_counts()

African American         10369
White                     5198
Other                      214
Asian                      205
Unknown                     67
Hispanic Latino/White       51
Hispanic Latino/Black       22
East Indian                 16
Pacific Island               6
American Indian              5
Name: Medview_Race, dtype: int64

In [5]:
df1PatientList = set(df1[df1.Medview_Race=='White']['DummyID'].values)

df2PatientList = set(df2['DummyID'].values)

In [6]:
q1 = df1PatientList.intersection(df2PatientList)

In [7]:
q2 = set(df1.DummyID).intersection(set(df2.DummyID.values))

In [8]:
extraPatients = list(q2.difference(q1))

In [16]:
len(df1[df1.DummyID.isin(extraPatients)]['DummyID'].values)

22

In [13]:
df2[df2.DummyID.isin(extraPatients)].sort_values('DummyID')

,DummyID,ScreenDate,Medview_Race,Age,Density_Overall,BMI
284,4121127,11/21/2011,White,42.4,3.0,32.0
716,5127892,11/30/2012,White,42.7,2.0,27.2
705,5129833,11/8/2012,White,47.5,2.0,31.1
889,5265592,1/18/2013,White,71.0,2.0,30.1
1333,75241874,4/23/2013,White,46.6,2.0,36.9
1430,75324130,5/31/2013,White,64.4,2.0,25.4
2093,75834114,11/27/2013,White,48.4,3.0,26.8
2196,75913620,1/20/2014,White,65.8,2.0,24.1
2313,76007543,1/31/2014,White,41.1,2.0,23.2
2593,76231025,4/18/2014,White,49.3,3.0,29.0


In [45]:

def createDataset2(n=600,seed = 123,full=False):

    patientList = []
    fullFileList = []

    inputFolder1 = '/home/santhosr/Documents/Birad/ProcessedData/FullRes'
    truthFile1 = '/home/santhosr/Documents/Birad/birad_targetFile.csv'

    inputFolder2 = '/home/santhosr/Documents/Birad/ProcessedData/PennExtra_3500/'
    truthFile2 = '/home/santhosr/Documents/Birad/RaceDL_ExtraCaucasian.csv'

    df1 = pd.read_csv('/home/santhosr/Documents/Birad/birad_targetFile.csv')
    df1.drop(['PresIntentType','DBT'],inplace = True,axis=1)


    df2 = pd.read_csv('/home/santhosr/Documents/Birad/RaceDL_ExtraCaucasian.csv')
    df2.Medview_Race = 'White'
    
    ## Removing IDs from df2 which are already present in df1
    idList = list(df1.DummyID.values)
    df2 = df2[~df2.DummyID.isin(idList)]
    
    truth = pd.concat([df1,df2],sort=True)

    ## Reading from set 1
    for i in range(1,5):

        folder = os.path.join(inputFolder1,str(i))
        fileList = os.listdir(folder)
        fileList = [os.path.join('FullRes',str(i),x) for x in fileList]
        fullFileList = fullFileList + fileList
        print(len(fileList))

        patientList = patientList + [int(x.split("/")[-1].split("_")[0]) for x in fileList]
    
    patientList1 = patientList.copy()
    ## Reading from set 2
    print(len(patientList))
    
    fileList= os.listdir(inputFolder2)
    fileList = [os.path.join('PennExtra_3500',x) for x in fileList]
    d = pd.DataFrame(fileList)
    d[1] = d[0].apply(lambda x : int(x.split("/")[1].split("_")[0]))
    d = d[d[1].isin(df2.DummyID.values)]
    fileList = list(d[0].values)
    fullFileList += list(d[0].values)
    
    patientList += [int(x.split("/")[-1].split("_")[0]) for x in fileList]
    print(len(patientList))
    
    patientList2 = patientList.copy()
    
    #Retaining only the patients with 4 views
    k=pd.Series(patientList).value_counts().reset_index()
    patientList = k[k[0]==4]['index'].values
    print("total number of patients",len(patientList))

    patientList = np.array(list(set(patientList)))
    df = pd.DataFrame({'DummyID':patientList})
    df = pd.merge(df,truth, how='left')
    df1 = df1.copy()
    df = df.drop_duplicates(subset=['DummyID'])
    
    



    #Creates equal number of patients from White and AA groups
    white = df[df.Medview_Race=='White']
    AA = df[df.Medview_Race=='African American']

    white = white.sort_values('DummyID')
    AA = AA.sort_values('DummyID')

    #Randomly selects n patients from each group
    whiteOverallList = np.random.choice(white.DummyID, n,replace = False)
    AAOverallList = np.random.choice(AA.DummyID, n,replace = False)

    #Training datasets (List of patients for training)
    whiteTrainList = np.random.choice(whiteOverallList, int(0.8*n), replace = False)
    AATrainList = np.random.choice(AAOverallList, int(0.8*n), replace = False)

    #Validation datasets (List of patients for validation)
    whiteValidList = np.array(list(set(whiteOverallList).difference(set(whiteTrainList))))
    AAValidList = np.array(list(set(AAOverallList).difference(set(AATrainList))))

    trainPatientList = np.concatenate([whiteTrainList, AATrainList])
    validPatientList = np.concatenate([whiteValidList, AAValidList])

    temp = pd.DataFrame(fullFileList)
    temp.columns = ['filename']

    temp['DummyID'] = temp.filename.apply(lambda x : int(x.split("/")[-1].split("_")[0]))
                                          
    trainTemp = temp[temp.DummyID.isin(trainPatientList)]
    validTemp = temp[temp.DummyID.isin(validPatientList)]

    trainTemp['train'] = False
    validTemp['train'] = True

    df= pd.concat([trainTemp, validTemp], sort = True)

    #Shuffling data
    index = list(range(len(df)))
    np.random.shuffle(index)
    df = df.iloc[index]
                                          
    return df, fullFileList

In [46]:
df,f = createDataset2(seed=123,n=5000)

8405
30753
13438
816
53412
64527
total number of patients 16108


/home/santhosr/.conda/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:99: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/santhosr/.conda/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:100: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [44]:
df[df.DummyID==4077998]

,DummyID,filename,train
17787,4077998,FullRes/2/4077998_L_CC_1.jpg,False
29425,4077998,FullRes/2/4077998_R_CC_1.jpg,False
8512,4077998,FullRes/2/4077998_R_MLO_1.jpg,False
19460,4077998,FullRes/2/4077998_L_MLO_1.jpg,False


'PennExtra_3500/76251016_FOR-PROCESSING_R_CC_1.jpg'

In [41]:
patientList2 = list(set(patientList2).difference(set(patientList1)))

In [42]:
len(patientList2)

2782

### New Dataset

In [12]:
inputFolder2 = '/home/santhosr/Documents/Birad/ProcessedData/PennExtra_3500/'
truthFile2 = '/home/santhosr/Documents/Birad/RaceDL_ExtraCaucasian.csv'

fileList= os.listdir(inputFolder2)
fileList = [os.path.join('PennExtra_3500',x) for x in fileList]
# fullFileList += fileList

patientList = [int(x.split("/")[-1].split("_")[0]) for x in fileList]
print(len(patientList))

k=pd.Series(patientList).value_counts().reset_index()
patientList = k[k[0]==4]['index'].values
print("total number of patients",len(patientList))

patientList2 = patientList.copy()

13979
total number of patients 3489


In [20]:
'PennExtra_3500/4077998_FOR-PROCESSING_R_MLO_1.jpg' in fileList

True

In [21]:
'PennExtra_3500/4077998_FOR-PROCESSING_R_MLO_1.jpg' in fileList

True

In [22]:
inputFolder1 = '/home/santhosr/Documents/Birad/ProcessedData/FullRes'
truthFile1 = '/home/santhosr/Documents/Birad/birad_targetFile.csv'

patientList = []

for i in range(1,5):
    folder = os.path.join(inputFolder1,str(i))
    fileList = os.listdir(folder)
    fileList = [os.path.join('FullRes',str(i),x) for x in fileList]
    print(len(fileList))

    patientList = patientList + [int(x.split("/")[-1].split("_")[0]) for x in fileList]

k=pd.Series(patientList).value_counts().reset_index()
patientList = k[k[0]==4]['index'].values
print("total number of patients",len(patientList))

patientList1 = patientList.copy()

8405
30753
13438
816
total number of patients 13333


In [72]:
idList = list(df1.DummyID.values)

len(df2[~df2.DummyID.isin(idList)])

2782

In [73]:
len(df2)-2782

718

In [58]:
len(df1[df1.DummyID.isin(patientList1)][df1.Medview_Race=='White'] )

/home/santhosr/.conda/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


2411

In [57]:
len(df2[df2.DummyID.isin(patientList2)])

3489

In [59]:
2411+3489

5900

In [60]:
len(set(patientList1).intersection(set(patientList2)))

714

In [23]:
patientList = []
fullFileList = []

inputFolder1 = '/home/santhosr/Documents/Birad/ProcessedData/FullRes'
truthFile1 = '/home/santhosr/Documents/Birad/birad_targetFile.csv'

inputFolder2 = '/home/santhosr/Documents/Birad/ProcessedData/PennExtra_3500/'
truthFile2 = '/home/santhosr/Documents/Birad/RaceDL_ExtraCaucasian.csv'

df1 = pd.read_csv('/home/santhosr/Documents/Birad/birad_targetFile.csv')
df1.drop(['PresIntentType','DBT'],inplace = True,axis=1)


df2 = pd.read_csv('/home/santhosr/Documents/Birad/RaceDL_ExtraCaucasian.csv')
df2.Medview_Race = 'White'

## Removing IDs from df2 which are already present in df1
idList = list(df1.DummyID.values)
df2 = df2[~df2.DummyID.isin(idList)]

truth = pd.concat([df1,df2],sort=True)

## Reading from set 1
for i in range(1,5):

    folder = os.path.join(inputFolder1,str(i))
    fileList = os.listdir(folder)
    fileList = [os.path.join('FullRes',str(i),x) for x in fileList]
    fullFileList = fullFileList + fileList
    print(len(fileList))

    patientList = patientList + [int(x.split("/")[-1].split("_")[0]) for x in fileList]

patientList1 = patientList.copy()
## Reading from set 2
print(len(patientList))

fileList= os.listdir(inputFolder2)
fileList = [os.path.join('PennExtra_3500',x) for x in fileList]

d = pd.DataFrame(fileList)
d[1] = d[0].apply(lambda x : int(x.split("/")[1].split("_")[0]))
d = d[d[1].isin(df2.DummyID.values)]

fullFileList += list(d[0].values)

patientList += [int(x.split("/")[-1].split("_")[0]) for x in fileList]
print(len(patientList))

patientList2 = patientList.copy()

#Retaining only the patients with 4 views
k=pd.Series(patientList).value_counts().reset_index()
patientList = k[k[0]>=4]['index'].values
print("total number of patients",len(patientList))

8405
30753
13438
816
53412
67391
total number of patients 16112


In [25]:
len(fileList)

13979

In [30]:
d = pd.DataFrame(fileList)
d[1] = d[0].apply(lambda x : int(x.split("/")[1].split("_")[0]))
d = d[d[1].isin(df2.DummyID.values)]

In [31]:
d.head()

,0,1
0,PennExtra_3500/4002346_FOR-PROCESSING_L_MLO_1.jpg,4002346
1,PennExtra_3500/76323689_FOR-PROCESSING_R_CC_1.jpg,76323689
2,PennExtra_3500/75106103_FOR-PROCESSING_R_MLO_1...,75106103
3,PennExtra_3500/4690229_FOR-PROCESSING_L_MLO_1.jpg,4690229
4,PennExtra_3500/3755652_FOR-PROCESSING_L_CC_1.jpg,3755652


In [38]:
list(d[0].values)

['PennExtra_3500/4002346_FOR-PROCESSING_L_MLO_1.jpg',
 'PennExtra_3500/76323689_FOR-PROCESSING_R_CC_1.jpg',
 'PennExtra_3500/75106103_FOR-PROCESSING_R_MLO_1.jpg',
 'PennExtra_3500/4690229_FOR-PROCESSING_L_MLO_1.jpg',
 'PennExtra_3500/3755652_FOR-PROCESSING_L_CC_1.jpg',
 'PennExtra_3500/76609436_FOR-PROCESSING_R_MLO_1.jpg',
 'PennExtra_3500/4082563_FOR-PROCESSING_R_MLO_1.jpg',
 'PennExtra_3500/75869431_FOR-PROCESSING_R_MLO_1.jpg',
 'PennExtra_3500/75199428_FOR-PROCESSING_R_MLO_1.jpg',
 'PennExtra_3500/75319978_FOR-PROCESSING_R_CC_1.jpg',
 'PennExtra_3500/76421522_FOR-PROCESSING_L_MLO_1.jpg',
 'PennExtra_3500/75277656_FOR-PROCESSING_R_MLO_1.jpg',
 'PennExtra_3500/76916846_FOR-PROCESSING_L_CC_1.jpg',
 'PennExtra_3500/5183007_FOR-PROCESSING_R_MLO_1.jpg',
 'PennExtra_3500/75699172_FOR-PROCESSING_R_MLO_1.jpg',
 'PennExtra_3500/76912298_FOR-PROCESSING_R_MLO_1.jpg',
 'PennExtra_3500/76374773_FOR-PROCESSING_L_CC_1.jpg',
 'PennExtra_3500/76811510_FOR-PROCESSING_L_MLO_1.jpg',
 'PennExtra_3500/768

### Error while reading files

In [9]:
errorFiles = []



fullFileList = []

inputFolder1 = '/home/santhosr/Documents/Birad/ProcessedData/FullRes'
truthFile1 = '/home/santhosr/Documents/Birad/birad_targetFile.csv'

inputFolder2 = '/home/santhosr/Documents/Birad/ProcessedData/PennExtra_3500/'
truthFile2 = '/home/santhosr/Documents/Birad/RaceDL_ExtraCaucasian.csv'

df1 = pd.read_csv('/home/santhosr/Documents/Birad/birad_targetFile.csv')
df1.drop(['PresIntentType','DBT'],inplace = True,axis=1)


df2 = pd.read_csv('/home/santhosr/Documents/Birad/RaceDL_ExtraCaucasian.csv')
df2.Medview_Race = 'White'

## Removing IDs from df2 which are already present in df1
idList = list(df1.DummyID.values)
df2 = df2[~df2.DummyID.isin(idList)]

truth = pd.concat([df1,df2],sort=True)

## Reading from set 1
for i in range(1,5):

    folder = os.path.join(inputFolder1,str(i))
    fileList = os.listdir(folder)
    fileList = [os.path.join('FullRes',str(i),x) for x in fileList]
    fullFileList = fullFileList + fileList


for i in fullFileList:
    try:
        img=imread(os.path.join('/home/santhosr/Documents/Birad/ProcessedData/',i))
    except:
        errorFiles.append(i)

/home/santhosr/.conda/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:37: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.


In [11]:
len(errorFiles)

2

In [12]:
errorFiles

['FullRes/2/4077998_R_MLO_1.jpg', 'FullRes/2/4077998_L_MLO_1.jpg']